## Q1. Running Elastic 

```
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [3]:
!curl localhost:9200 > elastic_cluster_info.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   539  100   539    0     0  19177      0 --:--:-- --:--:-- --:--:-- 24500


In [6]:
import json

with open('elastic_cluster_info.json') as f:
    elastic_cluster_info = json.load(f)
    

In [8]:
elastic_cluster_info["version"]["build_hash"]

'42f05b9372a9a4a470db3b52817899b99a76ee73'

In [9]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [15]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course'])

## Q2. Indexing the data

In [13]:
from elasticsearch import Elasticsearch

In [14]:
es_client = Elasticsearch('http://localhost:9200') 

In [16]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [17]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/Users/liablefish/github/llm-zoomcamp/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:03<00:00, 269.36it/s]


## Q3. Searching

In [48]:
query = "How do I execute a command in a running docker container?"

In [49]:
from typing import Optional

def elastic_search(query, size: int = 1, course: Optional[str] = None):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            },
        },
    }
    if course is not None:
        search_query["query"]["bool"]["filter"] = {"term": {"course": course}}

    response = es_client.search(index=index_name, body=search_query)
    return response['hits']['hits']

In [50]:
elastic_search(query)

[{'_index': 'course-questions',
  '_id': 'U_30SJABFUvEoHZZcNKf',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}}]

## Q4. Filtering

In [51]:
results = elastic_search(
    query,
    size=3, 
    course="machine-learning-zoomcamp",
)

In [52]:
results[-1]

{'_index': 'course-questions',
 '_id': 'c_30SJABFUvEoHZZcdIC',
 '_score': 49.938507,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

## Q5. Building a prompt

In [53]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [54]:
context = "\n\n".join(
    context_template.format(question=doc["_source"]["question"], text=doc["_source"]["text"]) 
    for doc in results
)

In [55]:
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [56]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [57]:
prompt = prompt_template.format(question=query, context=context)

In [59]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [60]:
len(prompt)

1462

## Q6. Tokens

In [62]:
# pip install tiktoken

In [64]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [66]:
len(encoding.encode(prompt))

322

## Bonus: generating the answer (ungraded)

In [86]:
response = requests.post(
    "http://localhost:11434/api/generate", 
    json={
      "model": "llama3:8b",
      "prompt": prompt,
    },
)

In [94]:
outputs = [json.loads(text) for text in response.text.strip().split("\n")]

In [105]:
print("".join(out["response"] for out in outputs))

Based on the provided context, to execute a command in a running Docker container, you can use the following command:

`docker exec -it <container-id> bash`

Replace `<container-id>` with the actual ID of your running Docker container, which you can find using `docker ps`. This will open a new shell session within the running container.
